# Testing

In [ ]:
import sys
sys.path.append('c:\\ProgramData\\Anaconda3\\Lib\\site-packages\\deuces\\')

import pboard
import pcard
import pdeck
import phand
import pplayer
import ppot

import gc

In [ ]:
gc.collect()

In [ ]:
POT = ppot.Pot()
print(POT.show_pot())
POT.increase_pot(25)
print(POT.show_pot())

In [ ]:
card0 = pcard.Card('Ts')
card1 = pcard.Card('8D')
print(card0.show_card(), card1.show_card())
print(card0.figure(), card1.figure())
print(card0.color(), card1.color())
print(card0.value(), card1.value())

In [ ]:
DECK = pdeck.Deck()
print(len(DECK.show_deck()))
print(DECK.nth_card(13).show_card())
BOARD = DECK.make_board()
HAND = DECK.make_hand()
print(BOARD.show_board()[0].show_card(), BOARD.show_board()[1].show_card(), BOARD.show_board()[2].show_card(), BOARD.show_board()[3].show_card(), BOARD.show_board()[4].show_card())
print(HAND.show_hand()[0].show_card(), HAND.show_hand()[1].show_card())
print(len(DECK.show_deck()))
print(DECK.draw_by_number(13).show_card())
print(len(DECK.show_deck()))
print(DECK.draw_by_name('KD').show_card())
print(len(DECK.show_deck()))

In [ ]:
print(BOARD.show_board()[0].show_card(), BOARD.show_board()[1].show_card(), BOARD.show_board()[2].show_card(), BOARD.show_board()[3].show_card(), BOARD.show_board()[4].show_card())
print(BOARD.flop()[0].show_card(), BOARD.flop()[1].show_card(), BOARD.flop()[2].show_card())
print(BOARD.flop0().show_card(), BOARD.flop1().show_card(), BOARD.flop2().show_card(), BOARD.turn().show_card(), BOARD.river().show_card())

In [ ]:
print(HAND.show_hand()[0].show_card(), HAND.show_hand()[1].show_card())
print(HAND.best_five(BOARD))
print(HAND.hand_strength(BOARD))

In [ ]:
player2 = pplayer.Player(2, 100)
print(player2.number())
print(player2.general_name())
print(player2.show_player_hand())
player2.add_hand(HAND)
print(player2.show_player_hand().show_hand()[0].show_card(), player2.show_player_hand().show_hand()[1].show_card())
print(player2.chips())
player2.increase_chips(25)
print(player2.chips())
player2.decrease_chips(14)
print(player2.chips())
print(player2.position_nr())
print(player2.position_name())
player2.add_position(4)
print(player2.position_nr())
print(player2.position_name())

# Preparation

In [ ]:
# Create Pot
POT = ppot.Pot()

# Create Deck
DECK = pdeck.Deck()

# Create Board
BOARD = DECK.make_board()

# Create Players
player0 = pplayer.Player(0, 100)
player0.add_position(0)

player1 = pplayer.Player(1, 100)
player1.add_position(1)

player2 = pplayer.Player(2, 100)
player2.add_position(2)

player3 = pplayer.Player(3, 100)
player3.add_position(3)

player4 = pplayer.Player(4, 100)
player4.add_position(4)

player5 = pplayer.Player(5, 100)
player5.add_position(5)

# Create Hands
HAND0 = DECK.make_hand()
HAND1 = DECK.make_hand()
HAND2 = DECK.make_hand()
HAND3 = DECK.make_hand()
HAND4 = DECK.make_hand()
HAND5 = DECK.make_hand()

# Add Hands
player0.add_hand(HAND0)
player1.add_hand(HAND1)
player2.add_hand(HAND2)
player3.add_hand(HAND3)
player4.add_hand(HAND4)
player5.add_hand(HAND5)

# Game and Table Initialization

In [1]:
'''poker test'''

# pylint: disable=E1101, E1601

import sys
sys.path.append('c:\\ProgramData\\Anaconda3\\Lib\\site-packages\\deuces\\')

import pcard
import pboard
import pdeck
import phand
import pplayer
import ppot

In [9]:
# parameters

number_of_players = 6
starting_chips_amount = 1000
small_blind_amount = 10
big_blind_amount = 2 * small_blind_amount

In [10]:
# table initialization

def table_init(nr_of_players, starting_chips):
    '''players initialization'''

    pot = ppot.Pot()

    players = []
    for i in range(nr_of_players):

        players.append(pplayer.Player(i, starting_chips))
        players[i].add_position(i)

    return pot, players

TABLE = table_init(number_of_players, starting_chips_amount)
POT = TABLE[0]
PLAYERS = TABLE[1]

In [11]:
# game initialization

for i in range(len(PLAYERS)):

    PLAYERS[i].add_position((PLAYERS[i].position_nr() + 1) % len(PLAYERS))

DECK = pdeck.Deck()
BOARD = DECK.make_board()

HANDS = [DECK.make_hand() for i in range(len(PLAYERS))]
for i in range(len(PLAYERS)):

    PLAYERS[i].add_hand(HANDS[i])

PLAYER_ORDER = [0 for i in range(len(PLAYERS))]
for i in range(len(PLAYERS)):
    
    PLAYER_ORDER[PLAYERS[i].position_nr()] = PLAYERS[i]

PLAYER_MOVE = [1 for i in range(len(PLAYERS))]

In [12]:
# check

for i in range(len(PLAYERS)):
    print(PLAYER_ORDER[i].number(), PLAYER_ORDER[i].general_name(), PLAYER_ORDER[i].position_nr(), PLAYER_ORDER[i].position_name(), PLAYER_ORDER[i].chips())
    
print('pot: ', POT.show_pot())

5 player5 0 Small Blind 1000
0 player0 1 Big Blind 1000
1 player1 2 Under the Gun 1000
2 player2 3 Middle 1000
3 player3 4 Tail 1000
4 player4 5 Dealer 1000
pot:  0


In [13]:
# preflop phase

def small_blind(player, small_blind_amount, pot):
    '''small blind move'''
    
    if isinstance(player, pplayer.Player) and isinstance(pot, ppot.Pot):
        player.decrease_chips(small_blind_amount)
        pot.increase_pot(small_blind_amount)
        PLAYER_MOVE[0] = 0
    else:
        raise TypeError('Wrong type(s)')
    
    return None

def big_blind(player, big_blind_amount, pot):
    '''big blind move'''
    
    if isinstance(player, pplayer.Player) and isinstance(pot, ppot.Pot):
        player.decrease_chips(big_blind_amount)
        pot.increase_pot(big_blind_amount)
        PLAYER_MOVE[1] = 0
    else:
        raise TypeError('Wrong type(s)')

    return None

In [14]:
small_blind(PLAYER_ORDER[0], small_blind_amount, POT)
big_blind(PLAYER_ORDER[1], big_blind_amount, POT)

In [15]:
# check

for i in range(len(PLAYERS)):
    print(PLAYER_ORDER[i].number(), PLAYER_ORDER[i].general_name(), PLAYER_ORDER[i].position_nr(), PLAYER_ORDER[i].position_name(), PLAYER_ORDER[i].chips())
    
print('pot: ', POT.show_pot())

5 player5 0 Small Blind 990
0 player0 1 Big Blind 980
1 player1 2 Under the Gun 1000
2 player2 3 Middle 1000
3 player3 4 Tail 1000
4 player4 5 Dealer 1000
pot:  30
